<a href="https://colab.research.google.com/github/romiaprilian7406/sp500-fair-value/blob/main/notebooks/sp500_fv_dataset_clean.ipynb">Open in Colab</a>

# Import Libraries

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import time
import warnings
import os

warnings.filterwarnings('ignore')

# Ticker list Function

In [2]:
# Mengambil daftar ticker S&P 500 terbaru dari dataset publik
def get_sp500_tickers():
    try:
        url = 'https://raw.githubusercontent.com/datasets/s-and-p-500-companies/refs/heads/main/data/constituents.csv'
        df = pd.read_csv(url)
        # Mengubah format ticker (misal: BRK.B -> BRK-B) agar sesuai dengan yfinance
        tickers = [t.replace('.', '-') for t in df['Symbol'].tolist()]
        print(f"Berhasil mendapatkan {len(tickers)} ticker S&P 500")
        return tickers
    except Exception as e:
        print(f"Gagal mengambil ticker: {e}")
        return []

# Data Fetching Function

In [3]:
# Mengambil data fundamental mentah untuk satu ticker
# Prioritas: Balance Sheet (Audit) -> Fallback: stock.info (TTM/Recent)
def fetch_raw_data(ticker):
    try:
        stock = yf.Ticker(ticker)
        bs_data = {}

        # STRATEGI A: Ambil dari Balance Sheet (Prioritas)
        try:
            bs = stock.balance_sheet
            if not bs.empty:
                latest = bs.iloc[:, 0] # Ambil tahun terbaru

                bs_data['total_assets'] = latest.get('Total Assets')

                # Handle variasi nama akun untuk Equity
                bs_data['total_equity'] = (
                    latest.get('Total Stockholder Equity') or
                    latest.get('Stockholders Equity') or
                    latest.get('Total Equity') or
                    latest.get('Common Stockholders Equity') or
                    latest.get('Total Equity Gross Minority Interest')
                )

                bs_data['total_debt'] = (
                    latest.get('Total Debt') or
                    latest.get('Total Liabilities Net Minority Interest')
                )

                bs_data['total_cash'] = (
                    latest.get('Cash And Cash Equivalents') or
                    latest.get('Cash') or
                    latest.get('Cash Cash Equivalents And Short Term Investments') or
                    latest.get('Cash Financial')
                )
        except Exception:
            pass # Lanjut ke fallback jika Balance Sheet gagal

        # STRATEGI B: Ambil dari stock.info (Fallback & Market Data)
        try:
            info = stock.info
        except Exception:
            return None # Skip jika info tidak bisa diambil sama sekali

        # Filter Wajib: Harus punya Market Cap & Revenue
        target_mcap = info.get('marketCap')
        revenue = info.get('totalRevenue')

        if target_mcap is None or revenue is None:
            return None

        # PENYUSUNAN DICTIONARY
        data = {
            'ticker': ticker,
            'sector': info.get('sector', 'Unknown'),

            # Features (X)
            'revenue': revenue,
            'ebitda': info.get('ebitda'),
            'net_income': info.get('netIncomeToCommon') or info.get('netIncome'),

            # Balance Sheet Items (Prioritas BS -> Fallback Info)
            'total_assets': bs_data.get('total_assets') or info.get('totalAssets'),

            # Robust Equity Fetching
            'total_equity': (
                bs_data.get('total_equity') or
                info.get('totalStockholderEquity') or
                info.get('stockholderEquity') or
                info.get('totalEquity')
            ),

            'total_debt': bs_data.get('total_debt') or info.get('totalDebt'),
            'total_cash': bs_data.get('total_cash') or info.get('totalCash'),

            'free_cashflow': info.get('freeCashflow'),
            'operating_cashflow': info.get('operatingCashflow'),

            # Target Variable (y)
            'target_market_cap': target_mcap
        }

        return data

    except Exception:
        return None

# Simple Data Cleaning & Preprocessing

In [4]:
def clean_raw_dataset(df):
    df = df.copy()

    # 1. Drop Duplicates
    df = df.drop_duplicates(subset=['ticker'], keep='first')

    # 2. Type Casting & Handling 'None' Strings
    num_cols = ['target_market_cap', 'revenue', 'ebitda', 'net_income',
                'total_assets', 'total_equity', 'total_debt', 'total_cash',
                'free_cashflow', 'operating_cashflow']

    for col in num_cols:
        if col in df.columns:
            # force=True akan mengubah 'None', 'NaN', inf menjadi NaN
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # 3. Handle Infinite Values (Critical for ML)
    df = df.replace([np.inf, -np.inf], np.nan)

    # 4. Standardize Sector
    df['sector'] = df['sector'].fillna('Unknown').astype(str).str.strip().str.title()

    # 5. Strict Sanity Filter
    # Wajib ada data fundamental inti: Jika Revenue/Asset/MarketCap kosong, data dianggap rusak.
    subset_wajib = ['revenue', 'target_market_cap', 'total_assets']
    df = df.dropna(subset=subset_wajib)

    # 6. Pastikan nilai positif untuk log-transform nanti
    df = df[
        (df['revenue'] > 0) &
        (df['total_assets'] > 0) &
        (df['target_market_cap'] > 0)
    ]

    # 7. Reorder Columns
    # Memindahkan target variable ke kolom paling kanan
    cols = [c for c in df.columns if c != 'target_market_cap']
    cols.append('target_market_cap')
    df = df[cols]

    return df

# Main Execution

In [5]:
tickers = get_sp500_tickers()
data_list = []

for ticker in tqdm(tickers):
    stock_data = fetch_raw_data(ticker)
    if stock_data:
        data_list.append(stock_data)
    time.sleep(0.5)

Berhasil mendapatkan 503 ticker S&P 500


  0%|          | 0/503 [00:00<?, ?it/s]

ERROR:yfinance:HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: WBA"}}}


In [6]:
# Buat DataFrame
df_raw = pd.DataFrame(data_list)

if not df_raw.empty:
    # 1. Bersihkan Dataset (Fungsi ini butuh kolom 'ticker' lowercase)
    df = clean_raw_dataset(df_raw)

    print(f"Total Raw Data  : {len(df_raw)}")
    print(f"Total Clean Data: {len(df)}")

    # Preview
    print("\nContoh Data:")
    display(df.head())
else:
    print("Gagal mengumpulkan data atau data kosong")
    df_final = pd.DataFrame()

Total Raw Data  : 502
Total Clean Data: 502

Contoh Data:


,ticker,sector,revenue,ebitda,net_income,total_assets,total_equity,total_debt,total_cash,free_cashflow,operating_cashflow,target_market_cap
0,MMM,Industrials,24824999936,6.161000e+09,3400999936,3.986800e+10,3.842000e+09,1.365900e+10,5.600000e+09,-4.898750e+08,2540999936,91638939648
1,AOS,Industrials,3830099968,7.843000e+08,530500000,3.240000e+09,1.883500e+09,2.167000e+08,2.396000e+08,4.540375e+08,655600000,9245750272
2,ABT,Healthcare,43842998272,1.174700e+10,13925999616,8.141400e+10,4.766400e+10,1.502100e+10,7.616000e+09,6.067875e+09,9118999552,224345161728
3,ABBV,Healthcare,59643998208,2.951900e+10,2348000000,1.351610e+11,3.325000e+09,6.714400e+10,5.524000e+09,2.079900e+10,20860000256,402433474560
4,ACN,Technology,69672976384,1.222253e+10,7678432768,6.539490e+10,3.119545e+10,8.182866e+09,1.147873e+10,1.047711e+10,11474399232,155713175552


# Simple EDA

In [7]:
# SHAPE DATASET
rows, cols = df.shape
print(f"Dimensi Dataset")
print(f"Total Baris    : {rows}")
print(f"Total Kolom    : {cols}\n")

# INFORMASI
print(df.info(),"\n")

# Target variable di sini adalah 'label'
features_list = [col for col in df.columns if col != 'label']
print(f"Jumlah Fitur : {len(features_list)}")
print(f"Daftar Fitur : {features_list}\n")

# STATISTIK DESKRIPTIF FITUR NUMERIK
print(f"\nStatistik Deskriptif (Fitur Input)")
# Filter hanya numerik
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

try:
    # Tampilkan transpose agar mudah dibaca
    display(df[numeric_cols].describe().T.round(2))
except:
    print(df[numeric_cols].describe().T.round(2))

# MISSING VALUES REPORT
print(f"\nLaporan Missing Values:")
missing_count = df.isnull().sum()
missing_pct = (df.isnull().sum() / len(df)) * 100

missing_df = pd.DataFrame({
    'Jumlah Hilang': missing_count,
    'Persentase (%)': missing_pct.map('{:.2f}%'.format)
})

# Tampilkan hanya kolom yang ada missing value
missing_only = missing_df[missing_df['Jumlah Hilang'] > 0]
if not missing_only.empty:
    print(missing_only)
else:
    print("Tidak ada missing values (Data Bersih!)")

Dimensi Dataset
Total Baris    : 502
Total Kolom    : 12

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502 entries, 0 to 501
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ticker              502 non-null    object 
 1   sector              502 non-null    object 
 2   revenue             502 non-null    int64  
 3   ebitda              474 non-null    float64
 4   net_income          502 non-null    int64  
 5   total_assets        502 non-null    float64
 6   total_equity        502 non-null    float64
 7   total_debt          498 non-null    float64
 8   total_cash          502 non-null    float64
 9   free_cashflow       476 non-null    float64
 10  operating_cashflow  502 non-null    int64  
 11  target_market_cap   502 non-null    int64  
dtypes: float64(6), int64(4), object(2)
memory usage: 47.2+ KB
None 

Jumlah Fitur : 12
Daftar Fitur : ['ticker', 'sector', 'revenue', 'ebitda', 'net_i

,count,mean,std,min,25%,50%,75%,max
revenue,502.0,3.607067e+10,7.414976e+10,7.723950e+08,6.809425e+09,1.411000e+10,2.904350e+10,7.030610e+11
ebitda,474.0,7.636334e+09,1.810431e+10,-6.154000e+09,1.692794e+09,3.083397e+09,6.275500e+09,1.664370e+11
net_income,502.0,4.475750e+09,1.300939e+10,-1.016700e+10,7.734363e+08,1.530297e+09,3.500250e+09,1.242510e+11
total_assets,502.0,9.487212e+10,2.980306e+11,1.248020e+09,1.318668e+10,2.896205e+10,6.983275e+10,4.002814e+12
total_equity,502.0,2.203909e+10,5.088020e+10,-1.423100e+10,3.844496e+09,9.499500e+09,2.029100e+10,6.493680e+11
total_debt,498.0,2.005513e+10,4.157584e+10,4.530000e+05,3.474648e+09,8.469321e+09,1.913300e+10,4.543110e+11
total_cash,502.0,7.456578e+09,3.163555e+10,1.326000e+06,5.418695e+08,1.528644e+09,3.966500e+09,4.693170e+11
free_cashflow,476.0,2.910620e+09,7.464248e+09,-2.334387e+10,5.604750e+08,1.278857e+09,2.914100e+09,7.886225e+10
operating_cashflow,502.0,5.983488e+09,1.734503e+10,-1.197480e+11,1.296570e+09,2.542000e+09,5.272109e+09,1.514240e+11
target_market_cap,502.0,1.306591e+11,4.355701e+11,3.775303e+09,2.035813e+10,3.792886e+10,8.494519e+10,4.316598e+12



Laporan Missing Values:
               Jumlah Hilang Persentase (%)
ebitda                    28          5.58%
total_debt                 4          0.80%
free_cashflow             26          5.18%


# Export Dataset

In [8]:
file_name = 'sp500_fv_dataset.csv'
df.to_csv(file_name, index=False)
print(f"File tersimpan: {file_name}")

File tersimpan: sp500_fv_dataset.csv
